In [65]:
import pandas as pd
import mlbstatsapi

In [66]:
# create mlb instance
mlb = mlbstatsapi.Mlb()

In [5]:
schedule = mlb.get_scheduled_games_by_date("2024-07-30")

for game in schedule:
    print(game)

ScheduleGames(gamepk=746441, link=/api/v1.1/game/746441/feed/live)
ScheduleGames(gamepk=747005, link=/api/v1.1/game/747005/feed/live)
ScheduleGames(gamepk=745546, link=/api/v1.1/game/745546/feed/live)
ScheduleGames(gamepk=745060, link=/api/v1.1/game/745060/feed/live)
ScheduleGames(gamepk=746683, link=/api/v1.1/game/746683/feed/live)
ScheduleGames(gamepk=746926, link=/api/v1.1/game/746926/feed/live)
ScheduleGames(gamepk=745787, link=/api/v1.1/game/745787/feed/live)
ScheduleGames(gamepk=745149, link=/api/v1.1/game/745149/feed/live)
ScheduleGames(gamepk=746365, link=/api/v1.1/game/746365/feed/live)
ScheduleGames(gamepk=746763, link=/api/v1.1/game/746763/feed/live)
ScheduleGames(gamepk=745958, link=/api/v1.1/game/745958/feed/live)
ScheduleGames(gamepk=746197, link=/api/v1.1/game/746197/feed/live)
ScheduleGames(gamepk=747167, link=/api/v1.1/game/747167/feed/live)
ScheduleGames(gamepk=745389, link=/api/v1.1/game/745389/feed/live)
ScheduleGames(gamepk=745309, link=/api/v1.1/game/745309/feed/l

In [67]:
game = mlb.get_game(747005)
play_by_play = game.livedata.linescore
play_by_play

Linescore(scheduledinnings=9, innings=[LinescoreInning(num=1, ordinalnum='1st', home=LinescoreTeamScoreing(hits=2, errors=0, leftonbase=1, runs=1, iswinner=None), away=LinescoreTeamScoreing(hits=1, errors=0, leftonbase=1, runs=0, iswinner=None)), LinescoreInning(num=2, ordinalnum='2nd', home=LinescoreTeamScoreing(hits=0, errors=0, leftonbase=0, runs=0, iswinner=None), away=LinescoreTeamScoreing(hits=0, errors=0, leftonbase=0, runs=0, iswinner=None)), LinescoreInning(num=3, ordinalnum='3rd', home=LinescoreTeamScoreing(hits=5, errors=0, leftonbase=1, runs=3, iswinner=None), away=LinescoreTeamScoreing(hits=0, errors=0, leftonbase=0, runs=0, iswinner=None)), LinescoreInning(num=4, ordinalnum='4th', home=LinescoreTeamScoreing(hits=2, errors=0, leftonbase=1, runs=1, iswinner=None), away=LinescoreTeamScoreing(hits=1, errors=1, leftonbase=2, runs=1, iswinner=None)), LinescoreInning(num=5, ordinalnum='5th', home=LinescoreTeamScoreing(hits=1, errors=0, leftonbase=0, runs=1, iswinner=None), away=

In [84]:
def get_runners(runners):
    on = []
    return_dict = {}

    for individual_runner in runners:
        if individual_runner.movement.originbase: 
            on.append(individual_runner.movement.originbase)
            
    first = '1B' in on
    second = '2B' in on
    third = '3B' in on


    return_dict['first'] = first
    return_dict['second'] = second
    return_dict['third'] = third

    return return_dict

In [85]:
playbyplay = mlb.get_game_play_by_play(747005)

#for p in playbyplay
r = playbyplay.allplays[0].result # have result, 
atbat_dict = {"type": r.type, "rbi" : r.rbi, "away_score" : r.awayscore, "home_score" : r.homescore, "event_type" : r.eventtype, "out" : r.isout} # this gives us type = atBat which is what we want. also awayscore, homescore, event, eventtype, don't need desc, want isout
a = playbyplay.allplays[0].about
atbat_dict['top'] = a.istopinning
atbat_dict['inning'] = a.inning

m = playbyplay.allplays[0].matchup
atbat_dict['hitter_id'] = m.batter.id
atbat_dict['hitter_hand'] = m.batside.code
atbat_dict['pitcher_id'] = m.pitcher.id
atbat_dict['pitcher_hand'] = m.pitchhand.code

runners = get_runners(playbyplay.allplays[0].runners)
atbat_dict['first'] = runners['first']
atbat_dict['second'] = runners['second']
atbat_dict['third'] = runners['third']

atbat_dict

{'type': 'atBat',
 'rbi': 0,
 'away_score': 0,
 'home_score': 0,
 'event_type': 'field_out',
 'out': True,
 'top': True,
 'inning': 1,
 'hitter_id': 543807,
 'hitter_hand': 'R',
 'pitcher_id': 669203,
 'pitcher_hand': 'R',
 'first': False,
 'second': False,
 'third': False}

In [87]:
# get batter stats
stats = ['season']
groups = ['hitting']
params = {'season': 2024}

stat_dict = mlb.get_player_stats(543807, stats=stats, groups=groups, **params)
season_hitting_stat = stat_dict['hitting']['season']

normalize = ['runs', 'doubles', 'triples', 'homeruns', 'strikeouts',
             'baseonballs', 'hits', 'hitbypitch', 'caughtstealing',
             'stolenbases']
keep = ['avg', 'obp', 'slg', 'ops', 'stolenbasepercentage']

for split in season_hitting_stat.splits:
     for k, v in split.stat.__dict__.items():
        print(k, v)


gamesplayed 104
flyouts None
groundouts 122
airouts 93
runs 54
doubles 16
triples 1
homeruns 13
strikeouts 81
baseonballs 45
intentionalwalks 0
hits 86
hitbypitch 2
avg .226
atbats 380
obp .310
slg .376
ops .686
caughtstealing 0
stolenbases 13
stolenbasepercentage 1.000
groundintodoubleplay 14
groundintotripleplay None
numberofpitches 1610
plateappearances 430
totalbases 143
rbi 41
leftonbase 153
sacbunts 0
sacflies 2
babip .253
groundoutstoairouts 1.31
catchersinterference 1
atbatsperhomerun 29.23


In [71]:
for r in playbyplay.allplays:
    # print(r.runners)
    on = []
    
    if len(r.runners) > 1:
        for individual_runner in r.runners:
            if individual_runner.movement.originbase: 
                on.append(individual_runner.movement.originbase)

    print(on)
            

[]
[]
[]
[]
[]
['1B']
[]
['1B', '1B']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['1B']
['2B', '1B']
['3B', '2B', '1B']
['2B', '3B']
[]
[]
[]
['1B', '1B']
['3B']
['1B']
[]
[]
[]
['1B', '2B']
['1B']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['1B']
['1B']
[]
[]
[]
[]
['1B']
[]
['1B']
['2B', '1B']
[]
[]
[]
[]
['1B']
['2B', '1B']
[]
[]
[]
[]
[]
[]
['1B']
[]
[]
[]
[]
[]
